In [ ]:
import os
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import time
from tensorflow.keras.layers import Dense, Conv2D, Reshape, MaxPooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization

from IPython import display

### 2.1 Prepare the Dataset 

In [ ]:
import urllib
categories = [line.rstrip(b'\n') for line in urllib.request.urlopen('https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt')]
print(categories[:10])
category = 'candle'

In [ ]:
# Creates a folder to download the original drawings into.
# We chose to use the numpy format : 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

if not os.path.isdir('npy_files'):
    os.mkdir('npy_files')
    
url = f'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{category}.npy'  
urllib.request.urlretrieve(url, f'npy_files/{category}.npy')

images = np.load(f'npy_files/{category}.npy')
print(f'{len(images)} images to train on')

# You can limit the amount of images you use for training by setting :
train_images = images[:10000]
# You should also define a smaller subset of the images for testing..
# TODO

# Notice that this to numpy format contains 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

In [ ]:
def preprocess(images):
    batch_size = 32
    shuffle_size = 1000
    #Construct a tf.Data.Dataset object
    data = tf.data.Dataset.from_tensor_slices(images)
    data = data.map(lambda x: tf.cast(x, tf.float32))
    data= data.map(lambda x: x / 128 - 1)
    #Normalize: Bring the images’ values into a sensible range. (-1, 1)
    data= data.map(lambda x: tf.reshape(x, (28, 28, 1)))
    #The images come as (1,784) pixel arrays. You should make sure to reshape 28x28x1


    data.cache()
    data = data.shuffle(shuffle_size)
    data = data.batch(batch_size)
    data = data.prefetch(tf.data.AUTOTUNE)
    return data

### 2.2 The Model

In [ ]:
class Disciminator(tf.keras.Model):
    
    def __init__(self): 
        super().__init__()
        self._activation_function_name = 'relu'
        self._output_activation_function_name = 'sigmoid'

        self.convolutional_layer_1 = Conv2D(filters=24, kernel_size=3, padding='same', activation=self._activation_function_name) 
        self.convolutional_layer_2 = Conv2D(filters=24, kernel_size=3, padding='same', activation=self._activation_function_name) 
        self.pooling = MaxPooling2D(pool_size=2, strides=2)

        self.convolutional_layer_3 = Conv2D(filters=48, kernel_size=3, padding='same', activation=self._activation_function_name) 
        self.convolutional_layer_4 = Conv2D(filters=48, kernel_size=3, padding='same', activation=self._activation_function_name)
        self.global_pool = GlobalAveragePooling2D()

        self.out = Dense(1, activation=self._output_activation_function_name)

    def call(self, x, training): #need to add batchnormalization still 
        input = self.convolutional_layer_1(input)
        input = self.convolutional_layer_2(input)
        input = self.pooling(input)
        input = self.convolutional_layer_3(input)
        input = self.convolutional_layer_4(input)
        input = self.global_pool(input)
        input = self.out(input)
        return input

In [ ]:
class Generator(tf.keras.Model):
    
    def __init__(self): 
        super().__init__()


    def call(self, x, training): 

### 2.3 Training 

In each training step, the generator is fed with random noise and creates
images from it. The discriminator sees a batch of true images and a batch of the generated
images. The loss of the discriminator is based on how well the discriminator
detected fake images as fake and real images as real. The loss of the generator is estimated by how well the generator was able to
fool the discriminator. The more images the discriminator falsely classified as real, the better our generator works and the smaller the Binary Cross Entropy loss between the discriminator’s predictions and all labels as true=1

check footnotes 5-11 for tips